# Sample script to auto schedule test plans

### Description

- This script enables users to auto schedule one or more test plans.
- It takes a list of test plan IDs and schedules the test plans based on:
    1. The system that matches the system filter of the test plan.
    2. The system that has the required number of fixtures to run the test plan.
    3. The earliest available time slot considering the estimated duration of the test plan within a range of six months from today.
- To specify the number of fixtures required to run a test plan, users can add a custom property named `Number of fixtures` specifying the required fixture count. If this property is missing, invalid, or greater than 8, the script defaults the value to 1.

Note: To customize the scheduling algorithm, modify the [Actions](#Actions), [APIs](#APIs) and [Algorithm](#Algorithm)

## Imports

Import the necessary Python modules to execute the notebook. [**`nisystemlink.clients`**](https://github.com/ni/nisystemlink-clients-python) provides the predefined models and methods for `Test Plans`, `System` and `Asset` APIs. **`Scrapbook`** is used to run notebooks and record data for integration with the SystemLink Notebook Execution Service. **`Datetime`** and **`Typing`** are also used.

In [ ]:
from datetime import datetime, timedelta, timezone
from typing import Dict, List, Tuple, Any
import scrapbook as sb

from nisystemlink.clients.assetmanagement import AssetManagementClient
from nisystemlink.clients.assetmanagement.models import (
    Asset,
    QueryAssetsRequest,
)
from nisystemlink.clients.systems import SystemsClient
from nisystemlink.clients.systems.models import QuerySystemsRequest
from nisystemlink.clients.test_plan import TestPlanClient
from nisystemlink.clients.test_plan.models import (
    QueryTestPlansRequest,
    ScheduleTestPlansRequest,
    ScheduleTestPlanRequest,
    TestPlan,
)

## Parameters

- **`test_plan_ids`**: IDs of the test plans to be scheduled.

Parameters are also listed in the metadata for the **parameters cell**, along with their default values.  
The Notebook Execution services use that metadata to pass parameters to this notebook.  
To view the metadata:
- Select the code cell
- Click the **wrench icon** in the right panel.

### Sample metadata

```json
{
  "papermill": {
    "parameters": {
      "test_plan_ids": []
    }
  },
  "systemlink": {
    "namespaces": [],
    "parameters": [
      {
        "display_name": "Test Plan IDs",
        "id": "test_plan_ids",
        "type": "string[]"
      }
    ],
    "version": 1
  },
  "tags": ["parameters"]
}
```
For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work).

In [ ]:
# Add the test plan Ids as a list of strings here
test_plan_ids = [] 

## Constants

In [ ]:
# Max systems to be queried
MAX_SYSTEMS_COUNT = 1000 

# Max fixtures to be queried
MAX_FIXTURES_COUNT = 1000

# Max test plans to be queried
MAX_TEST_PLANS_COUNT = 10000

# Max test plans to be queried per query
MAX_TEST_PLANS_COUNT_PER_QUERY = 1000

# Default estimated duration of test plan = 1 day"
DEFAULT_TEST_PLAN_DURATION_IN_SECONDS =  24 * 60 * 60

_current_time = datetime.now(timezone.utc)
_end_time_six_months = _current_time + timedelta(days = 182)

start_time = _current_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
due_date_time = _end_time_six_months.strftime("%Y-%m-%dT%H:%M:%S.000Z")

## Python clients

Initialize `TestPlanClient`, `SystemsClient` and `AssetManagementClient` to access the respective APIs.

In [ ]:
system_client = SystemsClient()
asset_client = AssetManagementClient()
test_plan_client = TestPlanClient()

## APIs

Provides methods to query test plans, schedule test plans, query systems and query assets.

Get testplan details by it's ID. 

In [ ]:
def get_test_plan(test_plan_to_schedule_id) -> TestPlan:
    return test_plan_client.get_test_plan(test_plan_to_schedule_id)

Query systems that match the test plan's system filter.

In [ ]:
def query_systems(system_filter) -> List[Dict[str, Any]]:
    request = QuerySystemsRequest(
        take = MAX_SYSTEMS_COUNT,
        projection = "new(id,alias)",
        orderBy = "lastUpdatedTimeStamp DESC",
        filter = system_filter
    )

    response = system_client.query_systems(request)
    return response.data

Query fixtures linked to a system based on the system IDs.

In [ ]:
def query_fixtures(systems) -> List[Asset]:
    system_ids = ",".join(f'"{system["id"]}"' for system in systems)

    request = QueryAssetsRequest(
        take = MAX_FIXTURES_COUNT,
        filter = (
            'AssetType == "FIXTURE" && '
            f'(locations.Any(location => new[] {{{system_ids}}}.Contains(location.minionId)))'
        ),
        orderBy = "LAST_UPDATED_TIMESTAMP",
        descending = True,
    )

    response = asset_client.query_assets(request)
    return response.assets

Query test plans scheduled to systems across a timeframe (six months from today).

In [ ]:
def query_test_plans(systems) -> List[TestPlan]:
    system_conditions = [f'systemId == "{system["id"]}"' for system in systems]
    testplans_filter = (
        f"({' or '.join(system_conditions)}) and "
        f"(plannedStartDateTime <= \"{due_date_time}\" and "
        f"estimatedEndDateTime >= \"{start_time}\")"
    )

    all_test_plans = []
    continuation_token = None

    while len(all_test_plans) < MAX_TEST_PLANS_COUNT:
        query_request = QueryTestPlansRequest(
            filter = testplans_filter,
            take = MAX_TEST_PLANS_COUNT_PER_QUERY,
            descending = True,
            continuation_token = continuation_token
        )

        response = test_plan_client.query_test_plans(query_request)

        if response.test_plans:
            all_test_plans.extend(response.test_plans)
            continuation_token = response.continuation_token

            if len(response.test_plans) < MAX_TEST_PLANS_COUNT_PER_QUERY or continuation_token is None:
                break
        else:
            break

    return all_test_plans[:MAX_TEST_PLANS_COUNT]


Schedule a test plan on a given system, fixtures and at a specific time slot and returns the schedule status.

In [ ]:

def _is_schedule_successful(schedule_response) -> bool:
    return schedule_response.failed_test_plans is None

In [ ]:
def schedule_test_plan(test_plan_id, slot, test_plan, fixture_ids) -> bool:
    request = ScheduleTestPlansRequest(
        test_plans = [
            ScheduleTestPlanRequest(
                id = test_plan_id,
                planned_start_date_time = slot['planned_start_time'],
                estimated_end_date_time = slot['estimated_end_time'],
                system_id = slot['system_id'],
                fixture_ids = fixture_ids,
                estimated_duration_in_seconds = test_plan.estimated_duration_in_seconds
            )
        ],
        replace = True
    )

    result = test_plan_client.schedule_test_plans(request)
    return _is_schedule_successful(result)


## Algorithm

The algorithm finds the earliest available time slot to schedule a test plan that meets its resource requirements.

Convert fixtures and systems to dictionary (systemId as key and list of fixture ids as value).

In [ ]:
def get_system_fixtures_map(fixtures) -> Dict[str, List[str]]:
    system_fixtures_map = {}
    for fixture in fixtures:
        system_id = fixture.location.minion_id
        fixture_id = fixture.id
        
        if system_id not in system_fixtures_map:
            system_fixtures_map[system_id] = []
        
        system_fixtures_map[system_id].append(fixture_id)
    return system_fixtures_map

Convert test plans and systems to dictionary (systemId as key and test plan's planned start date time and estimated end date time as value).

In [ ]:
def organize_test_plans_by_system(test_plans, systems) -> Dict[str, TestPlan]:
    system_with_test_plans = {}

    # Populate the dictionary with test plans
    for test_plan in test_plans:
        system_id = test_plan.system_id
        test_plan_info = {
            'planned_start_time': test_plan.planned_start_date_time,
            'estimated_end_time': test_plan.estimated_end_date_time
        }
        if system_id not in system_with_test_plans:
            system_with_test_plans[system_id] = []
        system_with_test_plans[system_id].append(test_plan_info)

    # Ensure all systems are represented in the dictionary
    for system in systems:
        system_id = system['id']
        if system_id not in system_with_test_plans:
            system_with_test_plans[system_id] = []

    return system_with_test_plans

Sort test plans in the system by their estimated end times.

In [ ]:
def sort_and_filter_system_test_plans(
    system_with_test_plans: Dict[str, List[Dict[str, str]]]
) -> Dict[str, List[Dict[str, str]]]:

    current_date_time = datetime.fromisoformat(start_time.replace("Z", "+00:00"))
    
    for system_id, test_plans in system_with_test_plans.items():
        test_plans.sort(key = lambda test_plan: datetime.fromisoformat(test_plan['estimated_end_time'].replace("Z", "+00:00")))

        test_plans = [
            test_plan for test_plan in test_plans 
            if datetime.fromisoformat(test_plan['estimated_end_time'].replace("Z", "+00:00")) > current_date_time
        ]

        system_with_test_plans[system_id] = test_plans

    return system_with_test_plans

Find available time slots to schedule the test plan on all matching systems, and return the earliest one.

In [ ]:
def find_available_slots(
    system_with_test_plans: Dict[str, List[Dict[str, str]]],
    test_plan: TestPlan
) -> List[Dict[str, str]]:

    current_date_time = datetime.fromisoformat(start_time.replace("Z", "+00:00"))
    
    test_plan_duration = timedelta(
        seconds = test_plan.estimated_duration_in_seconds
            if test_plan.estimated_duration_in_seconds != None else DEFAULT_TEST_PLAN_DURATION_IN_SECONDS
    )

    available_slots = []

    for system_id, occupied_slots in system_with_test_plans.items():
        occupied_slots.sort(
            key = lambda occupied_slot: datetime.fromisoformat(occupied_slot['planned_start_time'].replace("Z", "+00:00"))
        )
        
        last_busy_end_time = current_date_time

        for slot in occupied_slots:
            start_time = datetime.fromisoformat(slot['planned_start_time'].replace("Z", "+00:00"))
            end_time = datetime.fromisoformat(slot['estimated_end_time'].replace("Z", "+00:00"))

            if start_time > last_busy_end_time + test_plan_duration:
                available_slots.append({
                    'system_id': system_id,
                    'planned_start_time': last_busy_end_time.isoformat(),
                    'estimated_end_time': (last_busy_end_time + test_plan_duration).isoformat()
                })
                break

            last_busy_end_time = max(last_busy_end_time, end_time)

        if last_busy_end_time + test_plan_duration < datetime.fromisoformat(due_date_time.replace("Z", "+00:00")):
            available_slots.append({
                'system_id': system_id,
                'planned_start_time': last_busy_end_time.isoformat(),
                'estimated_end_time': (last_busy_end_time + test_plan_duration).isoformat()
            })

    available_slots.sort(key = lambda available_slot: datetime.fromisoformat(available_slot['planned_start_time']))

    return available_slots

Get earliest available timeslots based on fixture availability in a system.

In [ ]:
def get_available_slot(
    available_slots,
    system_fixtures_map,
    number_of_fixtures_to_select
) -> Dict[str, Any]:

    if(number_of_fixtures_to_select == 0):
        return available_slots[0] if available_slots else {}
    
    for slot in available_slots:
        system_id = slot['system_id']
        fixture_ids = system_fixtures_map.get(system_id, [])
        
        if len(fixture_ids) >= number_of_fixtures_to_select:
            selected_fixture_ids = fixture_ids[:number_of_fixtures_to_select]
            slot['fixture_ids'] = selected_fixture_ids
            return slot
    
    return available_slots[0] if available_slots else {}

Take a list of time intervals (as tuples of datetime objects) and merge any overlapping intervals into non-overlapping ones.

In [ ]:
def merge_intervals(
    intervals: List[Tuple[datetime, datetime]]
) -> List[Tuple[datetime, datetime]]:

    if not intervals:
        return []

    intervals.sort()
    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        last_start, last_end = merged[-1]
        if current_start <= last_end:
            merged[-1] = (last_start, max(last_end, current_end))
        else:
            merged.append((current_start, current_end))
    
    return merged

Identify available time slots within a specified range by excluding the given busy intervals.

In [ ]:
def invert_intervals(
    intervals: List[Tuple[datetime, datetime]],
    start: datetime, end: datetime
) -> List[Tuple[datetime, datetime]]:

    available = []
    prev_end = start

    for busy_start, busy_end in intervals:
        if prev_end < busy_start:
            available.append((prev_end, busy_start))
        prev_end = max(prev_end, busy_end)

    if prev_end < end:
        available.append((prev_end, end))

    return available

Find the earliest time slot within a given time range where a test plan can reserve a system and fixtures without conflicting with other test plan schedules.

In [ ]:
def find_earliest_common_slot(
    fixtures: List[str],
    system: str,
    test_plans,
    number_of_fixtures_to_schedule: int,
    duration: timedelta,
    search_start: datetime,
    search_end: datetime
) -> Dict[str, Any]:

    system_busy = []
    fixture_busy = {slave: [] for slave in fixtures}

    for test_plan in test_plans:
        planned_start_time = (
           test_plan.planned_start_date_time
            if isinstance(test_plan.planned_start_date_time, datetime)
           else datetime.fromisoformat(test_plan.planned_start_date_time)
        )
        
        estimated_end_time = (
            test_plan.estimated_end_date_time
            if isinstance(test_plan.estimated_end_date_time, datetime)
            else datetime.fromisoformat(test_plan.estimated_end_date_time)
        )

        if test_plan.system_id == system and len(test_plan.fixture_ids) == 0:
            system_busy.append((planned_start_time, estimated_end_time))
        for fixture in test_plan.fixture_ids:
            if fixture in fixture_busy:
                fixture_busy[fixture].append((planned_start_time, estimated_end_time))

    system_busy = merge_intervals(system_busy)

    for fixture in fixtures:
        combined = fixture_busy[fixture] + system_busy
        fixture_busy[fixture] = merge_intervals(combined)

    fixture_available = {}
    for fixture in fixtures:
        fixture_available[fixture] = invert_intervals(
            fixture_busy[fixture],
            search_start,
            search_end
        )

    time_points = set()
    for intervals in fixture_available.values():
        for start, end in intervals:
            time_points.add(start)
            time_points.add(end)
    time_points = sorted(time_points)

    for time_point in time_points:
        available_fixtures = []
        for fixture, intervals in fixture_available.items():
            for interval_start, interval_end in intervals:
                if interval_start <= time_point and interval_end >= time_point + duration:
                    available_fixtures.append(fixture)
                    break

        if len(available_fixtures) >= number_of_fixtures_to_schedule:
            return {
                'planned_start_time': time_point,
                'estimated_end_time': time_point + duration,
                'fixture_ids': available_fixtures[:number_of_fixtures_to_schedule]
            }

    return {}

Find the earliest available time slot to schedule a test plan across all matching systems and their fixtures.

In [ ]:
def find_best_system(
    system_fixture_map: Dict[str, List[str]],
    test_plan, 
    number_of_fixtures_to_schedule: int,
    duration: timedelta,
    search_start: datetime,
    search_end: datetime
) -> Dict[str, Any]:

    best_option = None

    for system, fixtures in system_fixture_map.items():
        result = find_earliest_common_slot(
            fixtures,
            system,
            test_plan,
            number_of_fixtures_to_schedule,
            duration,
            search_start,
            search_end
        )

        if result != {}:
            if not best_option or result['planned_start_time'] < best_option['planned_start_time']:
                best_option = {
                    'system_id': system,
                    'planned_start_time': result['planned_start_time'],
                    'estimated_end_time': result['estimated_end_time'],
                    'fixture_ids': result['fixture_ids']
                }

    return best_option

## Actions

Validates the input and manages the execution flow.

Validate the number of test plan IDs.

In [ ]:
def validate_test_plan_ids(test_plan_ids) -> bool:
    if len(test_plan_ids) == 0:
        print("Required at least one test plan id.")
        return False
    if len(test_plan_ids) > 1000:
        print("Update limit exceeded: Only up to 1000 test plans can be updated at a time.")
        sb.glue("Update limit exceeded: Only up to 1000 test plans can be updated at a time.")
        return False
    return True

Get the number of fixtures to schedule.

In [ ]:
def get_number_of_fixtures(test_plan) -> int:
    _number_of_fixtures = test_plan.properties.get("Number of fixtures")

    return int(_number_of_fixtures) if (
        isinstance(_number_of_fixtures, str)
        and _number_of_fixtures.isdigit()
        and (1 <= int(_number_of_fixtures) <= 8)
    ) else 1

Fetch systems and fixtures for the given test plan.

In [ ]:
def fetch_systems_and_fixtures(test_plan) -> tuple:
    systems = query_systems(test_plan.system_filter)
    if systems is None:
        return None, None
    fixtures = query_fixtures(systems)
    system_fixtures_map = get_system_fixtures_map(fixtures)
    return systems, system_fixtures_map

Find the system with fixture availability based on the earliest available timeslot and then schedule the test plan.

In [ ]:
def find_and_schedule(
    test_plan_to_schedule,
    systems,
    system_fixtures_map,
    test_plans,
    number_of_fixtures_to_schedule,
    duration,
    start_date_time
) -> bool:

    # Try to find the best system.
    best_system = find_best_system(
        system_fixtures_map,
        test_plans,
        number_of_fixtures_to_schedule,
        duration,
        start_date_time,
        datetime.fromisoformat(due_date_time.replace("Z", "+00:00"))
    )

    if best_system:
        fixtureIds = best_system['fixture_ids']
        best_system['planned_start_time'] = best_system['planned_start_time'].isoformat()
        best_system['estimated_end_time'] = best_system['estimated_end_time'].isoformat()
        return schedule_test_plan(test_plan_to_schedule.id, best_system, test_plan_to_schedule, fixtureIds)

    # Fallback to available slot scheduling, as the required specifications are not met.
    system_with_test_plans = organize_test_plans_by_system(test_plans, systems)
    sorted_system_test_plans = sort_and_filter_system_test_plans(system_with_test_plans)
    available_slots_in_systems = find_available_slots(sorted_system_test_plans, test_plan_to_schedule)
    earliest_available_slot = get_available_slot(available_slots_in_systems, system_fixtures_map, number_of_fixtures_to_schedule)
    
    fixtureIds = system_fixtures_map.get(earliest_available_slot['system_id'], [])[:number_of_fixtures_to_schedule]
    return schedule_test_plan(test_plan_to_schedule.id, earliest_available_slot, test_plan_to_schedule, fixtureIds)

Process a single test plan from the list.

In [ ]:
scheduled_test_plans = []
scheduled_test_plan_names = []
unscheduled_test_plans = []

def process_test_plan(test_plan_id, test_plan_ids) -> None:
    test_plan_to_schedule = get_test_plan(test_plan_id)
    if test_plan_to_schedule is None:
        return False
    
    number_of_fixtures_to_schedule = get_number_of_fixtures(test_plan_to_schedule)
    
    # If the number of fixtures is invalid, skip scheduling
    if number_of_fixtures_to_schedule > 8:
        unscheduled_test_plans.append(test_plan_id)
        return False

    # Fetch systems and fixtures
    systems, system_fixtures_map = fetch_systems_and_fixtures(test_plan_to_schedule)
    if systems is None:
        unscheduled_test_plans.append(test_plan_id)
        return False

    # Query existing test plans
    test_plans = query_test_plans(systems)
    if test_plans is None:
        unscheduled_test_plans.append(test_plan_id)
        return False

    # Scheduling logic
    duration = timedelta(
        seconds = test_plan_to_schedule.estimated_duration_in_seconds or DEFAULT_TEST_PLAN_DURATION_IN_SECONDS
    )
        
    start_date_time = datetime.fromisoformat(start_time.replace("Z", "+00:00"))

    is_scheduled = find_and_schedule(
        test_plan_to_schedule,
        systems,
        system_fixtures_map,
        test_plans,
        number_of_fixtures_to_schedule,
        duration,
        start_date_time
    )
    
    if is_scheduled:
        scheduled_test_plan_names.append(test_plan_to_schedule.name)
        scheduled_test_plans.append(test_plan_id)
    else:
        unscheduled_test_plans.append(test_plan_id)

    return True


Schedule test plans from the provided list of test plan IDs.

In [ ]:
def schedule_test_plans(test_plan_ids) -> None:
    if not validate_test_plan_ids(test_plan_ids):
        return
    
    # Process each test plan
    for test_plan_id in test_plan_ids:
        process_test_plan(test_plan_id, test_plan_ids)

In [ ]:
schedule_test_plans(test_plan_ids)

## Executions page output

Send the output to the Execution Results page via Scrapbook.

In [ ]:
test_plan_names = ', '.join(scheduled_test_plan_names)
sb.glue("Total test plans", len(test_plan_ids))
sb.glue("Scheduled test plans", test_plan_names)

if len(unscheduled_test_plans) == 0:
    sb.glue("Unscheduled test plans",  "-")
else:
    sb.glue("Unscheduled test plans",  unscheduled_test_plans)

## Output

Print the script output.

In [ ]:
print("Total test plans:", len(test_plan_ids))
print("Scheduled test plans:", test_plan_names)
if len(unscheduled_test_plans) == 0:
    print("Unscheduled test plans: -")
else:
    print("Unscheduled test plans:", unscheduled_test_plans)

## Next Steps
Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as **`Test Plan Scheduler`**.